In [2]:
import argparse
import time
import json
import requests

In [3]:
# 爬取的主题
topic_ids = [19554298, 19552330, 19565652, 19580349, 
             19939299, 19555547, 19594551, 19552832, 
             19577377, 19552826, 19615452]

In [ ]:
def get_answers(topic_id):
    page_no = 0
    client = pymongo.MongoClient()
    db = client["zhihu"]
    while True:
        is_saved = db.saved_topics.find({"topic_id": topic_id, "page_no": page_no}).count()
        if is_saved:
            print("{} {} already saved".format(topic_id, page_no))
            page_no += 1
            continue
        print(topic_id, page_no)
        is_end = get_answers_by_page(topic_id, page_no)
        page_no += 1
        if is_end:
            break

In [7]:
def get_answers_by_page(topic_id, page_no):
    offset = page_no * 10
    url = "https://www.zhihu.com/api/v4/topics/" + str(topic_id) + "/feeds/essence?include=data%5B%3F(target.type%3Dtopic_sticky_module)%5D.target.data%5B%3F(target.type%3Danswer)%5D.target.content%2Crelationship.is_authorized%2Cis_author%2Cvoting%2Cis_thanked%2Cis_nothelp%3Bdata%5B%3F(target.type%3Dtopic_sticky_module)%5D.target.data%5B%3F(target.type%3Danswer)%5D.target.is_normal%2Ccomment_count%2Cvoteup_count%2Ccontent%2Crelevant_info%2Cexcerpt.author.badge%5B%3F(type%3Dbest_answerer)%5D.topics%3Bdata%5B%3F(target.type%3Dtopic_sticky_module)%5D.target.data%5B%3F(target.type%3Darticle)%5D.target.content%2Cvoteup_count%2Ccomment_count%2Cvoting%2Cauthor.badge%5B%3F(type%3Dbest_answerer)%5D.topics%3Bdata%5B%3F(target.type%3Dtopic_sticky_module)%5D.target.data%5B%3F(target.type%3Dpeople)%5D.target.answer_count%2Carticles_count%2Cgender%2Cfollower_count%2Cis_followed%2Cis_following%2Cbadge%5B%3F(type%3Dbest_answerer)%5D.topics%3Bdata%5B%3F(target.type%3Danswer)%5D.target.annotation_detail%2Ccontent%2Crelationship.is_authorized%2Cis_author%2Cvoting%2Cis_thanked%2Cis_nothelp%3Bdata%5B%3F(target.type%3Danswer)%5D.target.author.badge%5B%3F(type%3Dbest_answerer)%5D.topics%3Bdata%5B%3F(target.type%3Darticle)%5D.target.annotation_detail%2Ccontent%2Cauthor.badge%5B%3F(type%3Dbest_answerer)%5D.topics%3Bdata%5B%3F(target.type%3Dquestion)%5D.target.annotation_detail%2Ccomment_count&limit=10&offset=" + str(page_no)
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36",
    }
    r = requests.get(url, verify=False, headers=headers)
    content = r.content.decode("utf-8")
    data = json.loads(content)
    is_end = data["paging"]["is_end"]
    items = data["data"]
    answer_ids = []
    if len(items) > 0:
        for item in items:
            item_type = item["target"]["type"]
            if item_type != "answer":
                continue
            question = item["target"]["question"]["title"]
            answer = item["target"]["content"]
            vote_num = item["target"]["voteup_count"]
            if vote_num < 1000:
                continue
            answer_id = item["target"]["id"]
            if answer_id in answer_ids:
                continue
            url = item["target"]["url"]
            print("=" * 50)
            print("Q: {}\nA: {}\nvote: {}".format(question, answer, vote_num))
            answer_ids.append(answer_id)
    return is_end

In [10]:
def query():
    client = pymongo.MongoClient()
    db = client["zhihu"]
    items = db.answers.aggregate([
        {"$addFields": {"answer_len": {"$strLenCP": "$target.content"}}},
        {"$match": {"answer_len": {"$lte": 50}}},
    ])
    answer_ids = []
    for item in items:
        item_type = item["target"]["type"]
        if item_type != "answer":
            continue
        question = item["target"]["question"]["title"]
        answer = item["target"]["content"]
        vote_num = item["target"]["voteup_count"]
        if vote_num < 1000:
            continue
        answer_id = item["target"]["id"]
        if answer_id in answer_ids:
            continue
        url = item["target"]["url"]
        print("=" * 50)
        print("\n")
        print("Q: {}\nA: {}\nvote: {}".format(question, answer, vote_num))
        answer_ids.append(answer_id)

In [11]:
get_answers_by_page(19554298, 0)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Q: 编程到底难在哪里？
A: <p>普通人：</p><p>我今天要买一斤苹果。</p><p><br></p><p><br></p><p>程序员：</p><p>我今天要买一斤苹果。</p><p>因为我只喜欢红富士苹果，所以我只买红富士苹果。</p><p>我能接受的最高价格是10元/斤。</p><p>正常情况下一斤苹果用一个袋子能装下，但是为防万一，我会带两个袋子。</p><p>我知道附近的3家水果店，所以我会依次访问这3家水果店。</p><p>根据上述条件，我设计出以下的买苹果的流程：</p><div class="highlight"><pre><code class="language-text"><span></span>买苹果流程开始
    对水果店0、水果店1、水果店2依次执行：
    拜访一家水果店流程开始
        走到此水果店
        如果此水果店没有开门，则结束当前的“拜访一家水果店流程”
        如果此水果店没有苹果，则结束当前的“拜访一家水果店流程”
        如果此水果店的苹果当中没有红富士苹果，则结束当前的“拜访一家水果店流程”
        如果此水果店的红富士苹果剩余不到一斤，则结束当前的“拜访一家水果店流程”
        如果此水果店的红富士苹果的价格高于10元/斤，则执行3次：
        讲价流程开始
            询问店主是否愿意将价格降到10元/斤或更低
            如果店主愿意，则跳过剩余的“讲价流程”
        讲价流程结束
        如果此水果店的红富士苹果的价格仍然高于10元/斤，则结束当前的“拜访一家水果店流程”
        打开一个袋子，将其作为当前的袋子
        重复执行以下流程，直到总重量大于一斤：
        装袋一个苹果流程开始
            从所有的不在袋子中的红富士苹果中选出最好的一个
            如果此苹果能装入当前的袋子，则将此苹果装入当前的袋子，否则执行：
            换袋子流程开始
                如果我有剩余的袋子，则从中任意选出一个并作为当前的袋子，否则执行：
                向店主要袋子流程开始
                   

False